In [3]:
import boto3
import json

In [7]:
region = 'ENTER YOUR REGION HERE'
kendra_index_id = 'ENTER YOUR KENDRA INDEX ID HERE'
TEXT_GENERATION_MODEL_ENDPOINT_NAME = 'ENTER YOUR TEXT GENERATION MODEL END POINT NAME HERE'

#TEXT_GENERATION_MODEL_ENDPOINT_NAME = 'jumpstart-dft-hf-llm-falcon-7b-instruct-bf16'

In [8]:
sagemaker_runtime_client = boto3.client('runtime.sagemaker',
                            region_name=region)

kendra_client = boto3.client('kendra',
                            region_name=region)

In [9]:
def get_kendra_results(query):
    result = ''
    context = []

    response = kendra_client.query(QueryText= query, IndexId=kendra_index_id)

    first_result_type = ''
    if response['TotalNumberOfResults']!=0:
        first_result_type = response['ResultItems'][0]['Type']
        
    if first_result_type == 'QUESTION_ANSWER' or first_result_type == 'ANSWER':
        result = response['ResultItems'][0]['DocumentExcerpt']['Text']
    
    elif first_result_type == 'DOCUMENT':
        for query_result in response["ResultItems"]:
            if query_result["Type"]=="DOCUMENT":
                answer_text = query_result["DocumentExcerpt"]["Text"]
                context.append(answer_text)
            
            for text in context:
                result += text
    
    return result

In [18]:
query = "What are the 5 pillars of well architected framework?"
context = get_kendra_results(query)

In [21]:
context

'Based on what we have learned, we have created the AWS Well-Architected Frame-\nwork, which provides a consistent set of best practices for customers and partners to\nevaluate architectures, and provides a set of questions you can use to evaluate how\nwell an architecture is aligned to AWS best practic'

In [22]:
template = """
        You are a helpful, polite, fact-based agent.
        If you don't know the answer, just say that you don't know.
        Please answer the following question using the context provided. 

        CONTEXT: 
        {context}
        =========
        QUESTION: {question} 
        ANSWER: """

In [23]:
prompt = template.format(context=context, question=query)
print(prompt)


        You are a helpful, polite, fact-based agent.
        If you don't know the answer, just say that you don't know.
        Please answer the following question using the context provided. 

        CONTEXT: 
        Based on what we have learned, we have created the AWS Well-Architected Frame-
work, which provides a consistent set of best practices for customers and partners to
evaluate architectures, and provides a set of questions you can use to evaluate how
well an architecture is aligned to AWS best practic
        QUESTION: What are the 5 pillars of well architected framework? 
        ANSWER: 


In [24]:
response_model = sagemaker_runtime_client.invoke_endpoint(
    EndpointName=TEXT_GENERATION_MODEL_ENDPOINT_NAME,
    Body=json.dumps(
        {"inputs": prompt, "parameters": {"max_new_tokens": 500}}
    ),
    ContentType="application/json",
)
response = json.loads(response_model["Body"].read())


In [25]:
print(response[0]["generated_text"])

1. Security: Implementing security measures to protect data and systems. 2.
Availability: Ensuring that systems and services are available and reliable. 3.
Performance: Optimizing the speed and efficiency of systems and services. 4.
Reliability: Ensuring that systems and services are reliable and can be counted on. 5.
Cost: Implementing cost-effective solutions to reduce the cost of running systems and services.
